<a href="https://colab.research.google.com/github/Alejandroll100/Alejandro-Lopez---Processamentos-TCC2/blob/main/Pos_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Manter ativa a sessão


In [ ]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

##Conexão com Google Drive

In [ ]:
from google.colab import drive

# Monta o Google Drive em /content/drive
drive.mount('/content/drive')

Mounted at /content/drive


##Pacotes e bibliotecas

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 129.5 MB/s eta 0:00:00


##Funções varias


In [ ]:
!nvidia-smi

Thu Nov 20 09:14:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:



!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn




  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 97.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 103.0 MB/s eta 0:00:00


##Reprojetar de Sistema Geográfico para Projetado

In [ ]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

def reproject_to_epsg(image_path: str,
                      target_epsg: int,
                      resampling_method: str = "nearest",
                      compress: bool = False) -> str:
    """
    Reprojeta um raster (presumivelmente lon/lat) para um CRS especificado por EPSG
    e salva na mesma pasta acrescentando '_{EPSG}_UTM' ao nome do ficheiro.

    Parâmetros
    ----------
    image_path : str
        Caminho completo para o GeoTIFF de entrada.
    target_epsg : int
        Código EPSG de destino (ex.: 31983, 32721, 3857, etc).
    resampling_method : str, opcional
        Método de reamostragem: "nearest", "bilinear", "cubic", "average" ou "mode".
        Padrão = "nearest".
    compress : bool, opcional
        Se True, escreve o GeoTIFF de saída com compressão LZW.
        Padrão = False.

    Retorna
    -------
    out_path : str
        Caminho completo do ficheiro re-projetado salvo.
    """

    # mapeamento de nomes para enum Resampling
    resampling_map = {
        "nearest": Resampling.nearest,
        "bilinear": Resampling.bilinear,
        "cubic": Resampling.cubic,
        "average": Resampling.average,
        "mode": Resampling.mode
    }
    if resampling_method not in resampling_map:
        raise ValueError(f"resampling_method inválido: {resampling_method}")

    if not isinstance(target_epsg, int):
        raise ValueError("target_epsg deve ser um inteiro (por exemplo: 31983)")

    target_crs = CRS.from_epsg(target_epsg)

    # abre o raster de entrada
    with rasterio.open(image_path) as src:
        if src.crs is None:
            raise ValueError(
                "O raster de entrada não tem CRS definido. Antes de reprojetar, defina o CRS do arquivo (ex.: EPSG:4326)."
            )

        # calcula transformada e dimensões de saída
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )

        # prepara metadados de saída
        out_meta = src.meta.copy()
        out_meta.update({
            "crs": target_crs,
            "transform": transform,
            "width": width,
            "height": height
        })

        # adiciona compressão se solicitado
        if compress:
            out_meta.update({
                "compress": "LZW"
            })

        # monta caminho de saída (mesma pasta, sufixo _{EPSG}_UTM antes da extensão)
        base, ext = os.path.splitext(image_path)
        out_path = f"{base}_{target_epsg}_UTM{ext}"

        # reproject e escrita
        with rasterio.open(out_path, "w", **out_meta) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=resampling_map[resampling_method]
                )

    print(f"Reprojeção concluída para EPSG:{target_epsg}")
    print(f"Arquivo salvo em: {out_path}")
    return out_path

# -----------------------
# Exemplo de uso:
IMAGE_PATH = r"/content/drive/MyDrive/amostras_TCC2/S1A_IW_GRDH_1SDV_20251021T102101_20251021T102126_061522_07AEAE_6391_COG_tnr_Cal_TC.tif"
# Substitua pelo EPSG desejado:
TARGET_EPSG = 31979  # EPSG Destino
reprojected = reproject_to_epsg(IMAGE_PATH, TARGET_EPSG, resampling_method="nearest", compress=True)


Reprojeção concluída para EPSG:31979
Arquivo salvo em: /content/drive/MyDrive/amostras_TCC2/S1A_IW_GRDH_1SDV_20251021T102101_20251021T102126_061522_07AEAE_6391_COG_tnr_Cal_TC_31979_UTM.tif


##Inferência


In [ ]:
# -*- coding: utf-8 -*-
"""
01_inferencia_yolo.py
- Lê GeoTIFF SAR (usando a BANDA 2 quando disponível)
- (Opcional) aplica "stretch" (2–98) para inferência visual
- Executa inferência YOLO em tiles com sobreposição
- Aplica NMS global
- Salva imagem anotada (.jpg)
- Salva arquivo intermediário .npz com detecções e metadados geoespaciais

Saídas:
  - OUT_ANN_JPG: imagem anotada
  - OUT_DETS_NPZ: arquivo intermediário com xyxy, conf, cls, transform, crs etc.
"""

import os
import numpy as np
import cv2
import rasterio
from ultralytics import YOLO
import supervision as sv
from rasterio.transform import xy as transform_xy

# ================= CONFIG =================
MODEL_PATH   = r"/content/drive/MyDrive/amostras_TCC2/best_s/best.pt"
#IMAGE_PATH   = r"/content/drive/MyDrive/amostras_TCC2/S1A_IW_GRDH_1SDV_20250825T090557_20250825T090622_060690_078D56_7A40_recorte_003_tif_32622_UTM.tif"
#IMAGE_PATH = r"/content/drive/MyDrive/novas/S1A_IW_GRDH_1SDV_20251020T093931_20251020T093956_061507_07AE10_699F_COG_recorte_001_tif_31980_UTM.tif" #manaus
IMAGE_PATH = r"/content/drive/MyDrive/amostras_TCC2/S1A_IW_GRDH_1SDV_20251021T102101_20251021T102126_061522_07AEAE_6391_COG_tnr_Cal_TC_31979_UTM.tif"
OUT_ANN_JPG  = r"/content/outputs/01_inferencia_annotated_experimento_final.jpg"
OUT_DETS_NPZ = r"/content/outputs/01_deteccoes_intermediario.npz"

# Inferência tiled
TILE        = 320
OVERLAP     = 64
CONF_THRES  = 0.25
IOU_NMS     = 0.50

# Visual
USE_STRETCH = True
P_LOW, P_HIGH = 2, 98
# ==========================================


def read_tif_as_rgb_with_meta(path):
    """
    Lê o GeoTIFF e extrai a BANDA 2 (segunda banda) quando disponível.
    - se existir >=2 bandas: usa a banda de índice 1 (segunda banda)
    - se existir apenas 1 banda: usa a banda 0 (fallback)
    Replica a banda selecionada em 3 canais para produzir uma imagem RGB (H,W,3) em float32.
    Retorna: (image, transform, crs, s_range_raw, s_az_raw)
    """
    with rasterio.open(path) as src:
        img = src.read()  # shape = (bands, H, W)
        transform = src.transform
        crs = src.crs
        s_range_raw = abs(transform.a)  # passo X
        s_az_raw    = abs(transform.e)  # passo Y

    # Seleciona a BANDA 2 (índice 1) sempre que possível. Caso contrário usa banda 0.
    if img.shape[0] >= 2:
        band = img[1]  # segunda banda (BAND 2)
    else:
        band = img[0]  # fallback: primeira banda disponível

    # Normalizar tipo e replicar para 3 canais (RGB) para compatibilidade visual
    image = np.repeat(band[..., np.newaxis], 3, axis=-1).astype(np.float32)
    return image, transform, crs, s_range_raw, s_az_raw


def make_norm_and_stretch(image, p_low=2, p_high=98):
    """
    Normaliza e aplica stretch percentílico sobre a imagem (espera image float).
    Retorna (image_norm, image_stretch) ambos uint8 com 3 canais.
    """
    # Normalização linear para 0-255
    image_norm = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Calcula percentis usando a banda única (podemos colapsar canais já que são iguais)
    single_band = image[..., 0]
    p2, p98 = np.percentile(single_band, (p_low, p_high))
    if p98 - p2 < 1e-6:
        image_stretch = image_norm.copy()
    else:
        stretched = np.clip((single_band - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)
        image_stretch = np.repeat(stretched[..., np.newaxis], 3, axis=-1)

    return image_norm, image_stretch


def run_tiled_inference(model, img_for_infer, tile=320, overlap=64, imgsz=320, conf=0.25):
    H, W, _ = img_for_infer.shape
    all_xyxy, all_conf, all_cls = [], [], []
    step = tile - overlap
    ys = list(range(0, max(H - tile, 0) + 1, step))
    xs = list(range(0, max(W - tile, 0) + 1, step))
    if ys and ys[-1] + tile < H: ys.append(H - tile)
    if xs and xs[-1] + tile < W: xs.append(W - tile)

    for y in ys:
        for x in xs:
            tile_img = img_for_infer[y:y+tile, x:x+tile, :]
            res = model.predict(source=tile_img, imgsz=imgsz, conf=conf, verbose=False)[0]
            if res.boxes is None or len(res.boxes) == 0:
                continue
            xyxy = res.boxes.xyxy.cpu().numpy().copy()
            confs = res.boxes.conf.cpu().numpy().copy()
            clss  = res.boxes.cls.cpu().numpy().astype(int)
            # desloca p/ coords globais
            xyxy[:, [0, 2]] += x
            xyxy[:, [1, 3]] += y
            all_xyxy.append(xyxy); all_conf.append(confs); all_cls.append(clss)

    if len(all_xyxy) == 0:
        return np.empty((0,4)), np.empty((0,)), np.empty((0,), dtype=int)

    xyxy = np.concatenate(all_xyxy, axis=0)
    confs = np.concatenate(all_conf, axis=0)
    clss  = np.concatenate(all_cls, axis=0)
    return xyxy, confs, clss


def main():
    os.makedirs(os.path.dirname(OUT_ANN_JPG), exist_ok=True)
    os.makedirs(os.path.dirname(OUT_DETS_NPZ), exist_ok=True)

    # Carrega modelo e imagem
    model = YOLO(MODEL_PATH)
    image_f32, transform, crs, s_range_raw, s_az_raw = read_tif_as_rgb_with_meta(IMAGE_PATH)
    H, W, _ = image_f32.shape

    # Entrada para inferência
    image_norm, image_stretch = make_norm_and_stretch(image_f32, P_LOW, P_HIGH)
    img_infer = image_stretch if USE_STRETCH else image_norm

    # Inferência tiled
    xyxy, confs, clss = run_tiled_inference(model, img_infer,
                                            tile=TILE, overlap=OVERLAP, imgsz=TILE, conf=CONF_THRES)

    # NMS global
    if xyxy.shape[0] > 0:
        det = sv.Detections(xyxy=xyxy, confidence=confs, class_id=clss).with_nms(threshold=IOU_NMS)
    else:
        det = sv.Detections.empty()

    # Visualização (anotar e salvar)
    vis = (image_stretch if USE_STRETCH else image_norm).copy()
    box_annotator = sv.BoxAnnotator(thickness=1)
    label_annotator = sv.LabelAnnotator(text_scale=0.4, text_thickness=1)
    names = model.names
    labels = []
    if len(det) > 0:
        for cid, c in zip(det.class_id, det.confidence):
            cname = names[int(cid)] if isinstance(names, (list, tuple, dict)) else str(int(cid))
            if isinstance(names, dict):
                cname = names.get(int(cid), str(int(cid)))
            labels.append(f"{cname} {c:.2f}")
        vis = box_annotator.annotate(vis, det)
        vis = label_annotator.annotate(vis, det, labels=labels)

    cv2.imwrite(OUT_ANN_JPG, cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))
    print(f"[OK] Anotado: {OUT_ANN_JPG} | Detecções pós-NMS: {len(det)}")

    # Serializa metadados mínimos para próximo estágio
    # (armazenamos transform como 6 números; crs em WKT se disponível)
    transform_arr = np.array([transform.a, transform.b, transform.c,
                              transform.d, transform.e, transform.f], dtype=np.float64)
    crs_wkt = str(crs.to_wkt()) if (crs is not None and hasattr(crs, "to_wkt")) else ""

    np.savez(
        OUT_DETS_NPZ,
        xyxy = det.xyxy.astype(np.float32) if len(det) else np.empty((0,4), dtype=np.float32),
        conf = det.confidence.astype(np.float32) if len(det) else np.empty((0,), dtype=np.float32),
        cls  = det.class_id.astype(np.int32) if len(det) else np.empty((0,), dtype=np.int32),
        H = H, W = W,
        transform = transform_arr,
        crs_wkt = crs_wkt,
        image_path = IMAGE_PATH
    )
    print(f"[OK] Intermediário salvo: {OUT_DETS_NPZ}")


if __name__ == "__main__":
    main()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/yolov12/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
[OK] Anotado: /content/outputs/01_inferencia_annotated_experimento_final.jpg | Detecções pós-NMS: 9
[OK] Intermediário salvo: /content/outputs/01_deteccoes_intermediario.npz


# Estimativa de atributos

In [ ]:
# -*- coding: utf-8 -*-
"""
02_estimar_atributos_min.py
Versão mínima: apenas Otsu + LSQ.
Dependências: numpy, rasterio, opencv-python-headless (cv2), scikit-image
Uso: adaptar caminhos IN_DETS_NPZ e OUT_DIR
"""
import os
import numpy as np
import cv2
from math import cos, radians
from skimage.filters import threshold_otsu
import rasterio
from rasterio.transform import xy as transform_xy

# ---------- CONFIG (ajuste caminhos) ----------
IN_DETS_NPZ = r"/content/outputs/01_deteccoes_intermediario.npz"
OUT_DIR     = r"/content/outputs"
OUT_CSV     = os.path.join(OUT_DIR, "atributos_deteccoes.csv")
DIR_OTSU    = os.path.join(OUT_DIR, "otsu")  # guarda mask_raw_{i}.npy e mask_vis_{i}.jpg

# ---------- utilitários simples ----------
EPS_W = 1e-9

def degrees_to_meters_scale(lat_deg: float):
    """Retorna (m_por_grau_long, m_por_grau_lat) aproximados."""
    m_per_deg_lat = 111_132.954
    m_per_deg_lon = 111_132.954 * cos(radians(lat_deg))
    return m_per_deg_lon, m_per_deg_lat

def otsu_puro(arr_uint8):
    """Aplica Otsu em imagem 2D (uint8) e retorna máscara binária (0/1 uint8)."""
    thr = float(threshold_otsu(arr_uint8))
    return (arr_uint8 >= thr).astype(np.uint8)

def medidas_lsq_centrada(mask_bin, s_range_m, s_az_m, min_pts=5):
    """
    LSQ centrada: ajusta reta aos pontos positivos da máscara no espaço (m).
    Retorna (comprimento_m, largura_m, extremos_len_pix, extremos_wid_pix)
    onde extremos_* são tuplas (x1,y1,x2,y2) em coordenadas de pixels do crop.
    """
    ys, xs = np.where(mask_bin > 0)
    if ys.size < min_pts:
        return None

    # converte para metros
    X = xs.astype(np.float32) * s_range_m
    Y = ys.astype(np.float32) * s_az_m

    # linha principal em metros (vx,vy direção, x0,y0 ponto)
    vx, vy, x0, y0 = cv2.fitLine(np.column_stack((X, Y)).astype(np.float32),
                                 cv2.DIST_L2, 0, 0.01, 0.01).ravel()
    vx, vy, x0, y0 = float(vx), float(vy), float(x0), float(y0)

    # versor ao longo da reta (u) e versor perpendicular (v) em metros
    u_norm = (vx**2 + vy**2) ** 0.5 + 1e-12
    ux, uy = vx / u_norm, vy / u_norm
    vx_m, vy_m = -uy, ux

    # projeções dos pontos sobre esses eixos (em metros)
    proj_u = (X - x0) * ux + (Y - y0) * uy
    proj_v = (X - x0) * vx_m + (Y - y0) * vy_m

    # comprimento = extensão ao longo do eixo u
    u_min, u_max = proj_u.min(), proj_u.max()
    comprimento_m = float(u_max - u_min)
    if comprimento_m <= 0:
        return comprimento_m, 0.0, None, None

    # largura = extensão ao longo do eixo v (usando todos os pontos)
    largura_m = float(proj_v.max() - proj_v.min())

    # achar pontos extremos em pixels (para desenhar)
    i_u_min = int(np.argmin(proj_u))
    i_u_max = int(np.argmax(proj_u))
    extremos_len_pix = (int(xs[i_u_min]), int(ys[i_u_min]), int(xs[i_u_max]), int(ys[i_u_max]))

    # extremos de largura centrados (projeção média)
    cx_m = 0.5 * (X[i_u_min] + X[i_u_max])
    cy_m = 0.5 * (Y[i_u_min] + Y[i_u_max])
    half_w = 0.5 * largura_m
    P1x_m = cx_m - vx_m * half_w
    P1y_m = cy_m - vy_m * half_w
    P2x_m = cx_m + vx_m * half_w
    P2y_m = cy_m + vy_m * half_w

    # converter de volta para pixels (aprox.)
    xW1 = int(round(P1x_m / s_range_m)); yW1 = int(round(P1y_m / s_az_m))
    xW2 = int(round(P2x_m / s_range_m)); yW2 = int(round(P2y_m / s_az_m))
    extremos_wid_pix = (xW1, yW1, xW2, yW2)

    return comprimento_m, largura_m, extremos_len_pix, extremos_wid_pix

def _draw_oriented_rectangle_simple(vis, extremos_len, extremos_wid, color=(0,0,255), thickness=2):
    """Desenha caixa orientada simples em imagem BGR (tudo em coordenadas de pixel do crop)."""
    if extremos_len is None or extremos_wid is None:
        return
    xL1, yL1, xL2, yL2 = extremos_len
    xW1, yW1, xW2, yW2 = extremos_wid
    Cx = 0.5 * (xL1 + xL2); Cy = 0.5 * (yL1 + yL2)
    dLx, dLy = (xL2 - xL1), (yL2 - yL1)
    Lh = 0.5 * (dLx**2 + dLy**2)**0.5
    if Lh < 1e-6:
        return
    nx = dLx / ( (dLx**2 + dLy**2)**0.5 )
    ny = dLy / ( (dLx**2 + dLy**2)**0.5 )
    vx, vy = -ny, nx
    Wp = 0.5 * ((xW2-xW1)**2 + (yW2-yW1)**2)**0.5
    corners = []
    for su in (+1, -1):
        for sv in (+1, -1):
            x = Cx + su * nx * Lh + sv * vx * Wp
            y = Cy + su * ny * Lh + sv * vy * Wp
            corners.append((int(round(x)), int(round(y))))
    poly = np.array([corners[0], corners[2], corners[3], corners[1]], dtype=np.int32)
    cv2.polylines(vis, [poly], isClosed=True, color=color, thickness=thickness)

# ---------- main ----------
def main():
    os.makedirs(DIR_OTSU, exist_ok=True)
    if not os.path.exists(IN_DETS_NPZ):
        raise FileNotFoundError(f"Intermediário não encontrado: {IN_DETS_NPZ}")

    d = np.load(IN_DETS_NPZ, allow_pickle=True)
    xyxy = d["xyxy"]; conf = d["conf"]; cls = d["cls"]
    H, W = int(d["H"]), int(d["W"])
    transform_arr = d["transform"]
    image_path = str(d["image_path"])
    transform = rasterio.Affine(*transform_arr)

    # abrir imagem para obter resolução (res_x,res_y) e CRS
    with rasterio.open(image_path) as src:
        img = src.read(1).astype(np.float32)  # só banda 0
        crs = src.crs
        try:
            res_x, res_y = src.res
            res_x = abs(res_x) if res_x != 0 else 1e-9
            res_y = abs(res_y) if res_y != 0 else 1e-9
        except Exception:
            res_x = abs(transform.a) if transform.a != 0 else 1e-9
            res_y = abs(transform.e) if transform.e != 0 else 1e-9

        # se geográfico, converte graus->metros (aprox.)
        def _is_geographic(_crs):
            try:
                return _crs and (_crs.is_geographic or _crs.to_string().upper() in ("EPSG:4326","WGS84"))
            except Exception:
                return False

        if _is_geographic(crs):
            # obter latitude aproximada do centro da imagem
            lon_c, lat_c = transform_xy(src.transform, H/2.0, W/2.0, offset='center')
            m_per_deg_lon, m_per_deg_lat = degrees_to_meters_scale(lat_c)
            s_range = float(max(res_x * m_per_deg_lon, 1e-9))
            s_az    = float(max(res_y * m_per_deg_lat, 1e-9))
        else:
            s_range = float(max(res_x, 1e-9))
            s_az    = float(max(res_y, 1e-9))

    rows = ["id,cls,conf,lon,lat,comprimento_m,largura_m,RA\n"]

    for i in range(xyxy.shape[0]):
        x1, y1, x2, y2 = xyxy[i].astype(int).tolist()
        c = int(cls[i]); p = float(conf[i])
        # pad mínimo 1 pixel para evitar bbox degenerate
        x1p, y1p = max(0, x1), max(0, y1)
        x2p, y2p = min(W-1, x2), min(H-1, y2)
        if x2p <= x1p or y2p <= y1p:
            continue

        crop = img[y1p:y2p, x1p:x2p].copy()
        # normalizar/uint8 simples para Otsu: mapear min->0 max->255 (evita NaN)
        arr = crop.copy()
        arr = np.nan_to_num(arr, nan=0.0)
        mn, mx = float(arr.min()), float(arr.max())
        if mx - mn < 1e-9:
            arr_u8 = np.zeros_like(arr, dtype=np.uint8)
        else:
            arr_u8 = np.clip(((arr - mn) * 255.0 / (mx - mn)), 0, 255).astype(np.uint8)

        # Otsu puro
        mask = otsu_puro(arr_u8)  # 0/1 uint8
        mask_path = os.path.join(DIR_OTSU, f"mask_raw_{i:04d}.npy")
        np.save(mask_path, mask)

        # LSQ sobre máscara no tamanho do crop
        comp = medidas_lsq_centrada(mask, s_range, s_az, min_pts=5)
        if comp is None:
            # escrever linha com zeros para indicar falha
            lon, lat = transform_xy(transform, (y1+y2)/2.0, (x1+x2)/2.0, offset='center')
            rows.append(f"{i},{c},{p:.4f},{lon:.6f},{lat:.6f},0,0,0\n")
            continue

        comprimento_m, largura_m, extremos_len_pix, extremos_wid_pix = comp
        RA = float(comprimento_m) / max(float(largura_m), EPS_W)

        # salvar visual simples (máscara colorida + caixa orientada) na resolução do crop
        vis = cv2.cvtColor((mask * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
        _draw_oriented_rectangle_simple(vis, extremos_len_pix, extremos_wid_pix, color=(0,0,255), thickness=2)
        vis_path = os.path.join(DIR_OTSU, f"mask_vis_{i:04d}.jpg")
        cv2.imwrite(vis_path, vis)

        # coordenadas lon/lat do centro do bbox (para o CSV)
        xc_pix, yc_pix = (x1 + x2)/2.0, (y1 + y2)/2.0
        lon, lat = transform_xy(transform, yc_pix, xc_pix, offset='center')

        rows.append(f"{i},{c},{p:.4f},{lon:.6f},{lat:.6f},{comprimento_m:.3f},{largura_m:.3f},{RA:.3f}\n")

    # salvar CSV final
    with open(OUT_CSV, "w") as f:
        f.writelines(rows)
    print("Processamento completo. Resultados em:", OUT_DIR)

if __name__ == "__main__":
    main()


Processamento completo. Resultados em: /content/outputs


# Exportar pontos vetoriais

In [ ]:
# -*- coding: utf-8 -*-
"""
03_exportar_shapefile.py (campos encurtados)
- Lê o CSV de atributos
- Cria GeoDataFrame com pontos nas coordenadas do IMAGE_PATH (lê o CRS do raster)
- Extrai início/fim de aquisição (UTC) do nome Sentinel-1 (IMAGE_PATH)
- Remove campos: cls, x1, y1, x2, y2, xc_pix, yc_pix
- Renomeia campos p/ <=10 chars (Shapefile)
- Salva Shapefile
"""
import os
import re
from datetime import datetime, timezone

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio

# ================= CONFIG =================
IN_CSV     = r"/content/outputs/atributos_deteccoes.csv"
OUT_SHP    = r"/content/outputs/outputs_atributos/deteccoes_pontos.shp"
REPROJECT_TO = None  # ex.: "EPSG:31983" ou None
IMAGE_PATH = IMAGE_PATH
# ==========================================

def parse_s1_datetimes(path: str):
    """Extrai (t0_utc, t1_utc) em ISO-8601 (com 'Z') do nome Sentinel-1."""
    fname = os.path.basename(path)
    m = re.search(r'_(\d{8}T\d{6})_(\d{8}T\d{6})_', fname)
    if not m:
        raise ValueError(
            f"Não foi possível extrair timestamps do nome: {fname}. "
            "Esperado algo como '_YYYYMMDDTHHMMSS_YYYYMMDDTHHMMSS_'."
        )
    s0, s1 = m.group(1), m.group(2)

    def to_iso_z(s):
        dt = datetime.strptime(s, "%Y%m%dT%H%M%S").replace(tzinfo=timezone.utc)
        return dt.isoformat().replace("+00:00", "Z")

    return to_iso_z(s0), to_iso_z(s1)

def main():
    if not os.path.exists(IN_CSV):
        raise FileNotFoundError(f"CSV não encontrado: {IN_CSV}")

    if not os.path.exists(IMAGE_PATH):
        raise FileNotFoundError(f"IMAGE_PATH não encontrado: {IMAGE_PATH}")

    # tempos da cena
    t0_iso, t1_iso = parse_s1_datetimes(IMAGE_PATH)

    # lê CSV
    df = pd.read_csv(IN_CSV)

    # garante colunas mínimas presentes no CSV original
    needed = ["lon", "lat", "comprimento_m", "largura_m", "conf"]
    for col in needed:
        if col not in df.columns:
            raise ValueError(f"Coluna ausente no CSV: {col}")

    # remove campos solicitados (ignora se não existirem)
    drop_cols = ["cls", "x1", "y1", "x2", "y2", "xc_pix", "yc_pix"]
    df = df.drop(columns=drop_cols, errors="ignore")

    # renomeia p/ caber no Shapefile (<=10 chars)
    rename_map = {
        "comprimento_m": "compr_m",
        "largura_m":     "larg_m",
    }
    df = df.rename(columns=rename_map)

    # adiciona tempos (constantes p/ a cena)
    df["t0_utc"] = t0_iso
    df["t1_utc"] = t1_iso

    # lê CRS do raster (IMAGE_PATH)
    with rasterio.open(IMAGE_PATH) as src:
        img_crs = src.crs  # rasterio CRS object or None
        if img_crs is None:
            raise ValueError(f"CRS inválido/nulo no raster: {IMAGE_PATH}")
        # converte para string aceitável pelo GeoPandas, ex.: "EPSG:32723" ou WKT
        try:
            img_crs_str = img_crs.to_string()
        except Exception:
            # fallback: use the CRS object directly
            img_crs_str = img_crs

    print(f"[INFO] CRS do raster detectado: {img_crs_str}")

    # construir geometria usando os pares (lon, lat) do CSV como coordenadas no CRS da imagem
    # OBS: aqui 'lon','lat' são interpretados como (x,y) já no sistema da imagem.
    # checar valores nulos
    if df["lon"].isnull().any() or df["lat"].isnull().any():
        n_null = int(df["lon"].isnull().sum() + df["lat"].isnull().sum())
        print(f"[WARN] Existem valores nulos em lon/lat no CSV ({n_null} entradas). Esses registros serão descartados.")
        df = df.dropna(subset=["lon", "lat"])

    # criar geometria (Point x,y) no CRS da imagem
    geometry = [Point(xy) for xy in zip(df["lon"].values, df["lat"].values)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=img_crs_str)

    # reprojeta se necessário
    if REPROJECT_TO is not None:
        print(f"[INFO] Reprojetando para: {REPROJECT_TO}")
        gdf = gdf.to_crs(REPROJECT_TO)

    # prepara pasta de saída
    os.makedirs(os.path.dirname(OUT_SHP), exist_ok=True)

    # salva shapefile
    # geopandas -> shapefile: alguns drivers exigem nomes de campo curtos; já renomeamos
    try:
        gdf.to_file(OUT_SHP, driver="ESRI Shapefile", encoding="utf-8")
    except Exception as e:
        # fallback: tentar sem encoding (compatibilidade)
        print(f"[WARN] falha ao salvar com encoding especificado: {e}. Tentando sem encoding.")
        gdf.to_file(OUT_SHP, driver="ESRI Shapefile")

    print(f"[OK] Shapefile salvo: {OUT_SHP} | {len(gdf)} features")
    print(f"t0_utc={t0_iso} | t1_utc={t1_iso}")
    print(f"[INFO] CRS salvo no Shapefile: {gdf.crs}")

if __name__ == "__main__":
    main()


[INFO] CRS do raster detectado: EPSG:31979
[OK] Shapefile salvo: /content/outputs/outputs_atributos/deteccoes_pontos.shp | 9 features
t0_utc=2025-10-21T10:21:01Z | t1_utc=2025-10-21T10:21:26Z
[INFO] CRS salvo no Shapefile: EPSG:31979


In [ ]:
from google.colab import files
import shutil

# Compactar a pasta (exemplo: "runs/detect/train")
shutil.make_archive("ouputs9", 'zip', "/content/outputs")

# Baixar o arquivo ZIP
files.download("ouputs9.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>